# STIS/FUV-MAMA custom extraction

In this notebook, we will learn how to use `rocky-worlds-utils` to process STIS/FUV-MAMA observations and perform a custom spectral extraction.

- Author: Leonardo Dos Santos (STScI)
- Date first published: November 19, 2025

## Imports

- `astropy`: FITS files I/O
- `matplotlib`: Plotting library
- `glob`: searching for files
- `numpy`: array calculations
- `rocky_worlds_utils`: various functions to facilitate data analysis

In [ ]:
from astropy.io import fits
from astroquery.mast import Mast, ObservationsClass
from astroquery.mast import Observations
import glob
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import numpy as np
from rocky_worlds_utils.hst import stis_analysis

%matplotlib inline
pylab.rcParams['figure.figsize'] = 15.0,9.0
pylab.rcParams['font.size'] = 18

The STIS spectrograph is one of the most versatile and well-calibrated instruments in the Hubble Space Telescope. However, it was not originally designed with observations of FUV-faint targets as a main driver. It turns out that the targets of the RWDDT program are very faint in the FUV, so the CALSTIS algorithm used to determine the position of the spectral trace in the flat-fielded exposures usually cannot properly find the stellar trace.

To add to the complications, the FUV-MAMA detector experiences high levels of dark current background when the detector is kept turned on for extended periods of time, which causes an uneven contamination accross the detector that needs to be carefully subtracted from the observations.

We shall deal with these two birds with only one stone: **a custom spectral extraction**.

The first step is to determine which exposures were obtained with the FUV-MAMA detector. We already queried and downloaded data for the target GJ 3929 in the notebook \#1 and stored them in the `example_data` folder. In this example, we are interested in those centered at the wavelength 1222 Angstrom.

In [ ]:
prefix = './example_data/'
flt_files = glob.glob(prefix + '*_flt.fits')

print('dataset\t\t detector\t grating\t cenwave')
for f in flt_files:
    header = fits.getheader(f, ext=0)
    print(header['ROOTNAME'] + '\t', header['DETECTOR'] + '\t', header['OPT_ELEM'] + '\t\t', header['CENWAVE'])

Looks like the datasets in question are `ofkb07020`, `ofkb07030`, `ofkb08010`, `ofkb08020` and `ofkb08030`. You might be wondering where `ofkb07010` is. Well, there was a guide star failure in Visit 7 that made the whole visit unusable, so we shall skip the analysis of these data and focus on those of Visit 8. There was also a guide star lock issue in the first orbit of Visit 8, so we shall ignore that one too. Thus, the only datasets we will work on are `ofkb08020` and `ofkb08030`.

The next step is to plot the respective flat-fielded, two-dimensional spectra to visually inspect the position of the stellar trace.

In [ ]:
datasets = ['ofkb08020', 'ofkb08030']

n_orbits = len(datasets)
fig, ax = plt.subplots(ncols=n_orbits, nrows=1, sharey=True)

for i in range(n_orbits):
    data = fits.getdata(prefix + datasets[i] + '_flt.fits')
    ax[i].imshow(data, origin='lower', vmin=0, vmax=5)
    ax[i].annotate(xy=(600, 950), text=datasets[i], color='white')
    ax[i].set_xlabel('Dipersion pixel')
    if i == 0:
        ax[i].set_ylabel('Cross-dispersion pixel')
plt.tight_layout()

The bright vertical emission near the 400-px in the dispersion direction is the geocoronal contamination, which we aim to eliminate. It looks like GJ 3929 is, indeed, a faint target in the FUV! But, with trained eyes, you may notice the faint smudge around 200-px in the cross-dispersion direction, near the geocoronal contamination: that is the Lyman-alpha emission of GJ 3929. Let's zoom in.

In the image computed in the cell below, we shall also plot, in units of pixel:
- The cross-dispersion position where we will extract the spectra (defined in CALSTIS as `a2center`). Upon visual inspection, the position is ~193 px for both datasets.
- The extraction width (defined in CALSTIS as `extrsize`; default value is 11 px for first-order spectra). The extraction of FUV-faint targets benefits from using larger apertures, so we shall increase the width of 15 px.
- The background offset (defined in CALSTIS as `bk1offst` and `bk2offst`), which is the position where we shall extract the background. For the purpose of this example, we will extract the background at an offset of 30 px, which overrides the default value of 300 px from CALSTIS. This is necessary to properly subtract the dark current background, which is more accurately estimated close to the stellar trace.
- The background size (defined in CALSTIS as `bk1size` and `bk2size`; default value is 5 px), which is the width of the background extraction region. Since there are two background regions, one above and one below the stellar trace, ideally we want to set the background size to half (rounded down) the size of the target extraction. So, in our example, that will be a width of 7 px.

In [ ]:
extract_width = 15
a2center = [193, 193]
bg_size = 7
bg_offset = 30

n_orbits = len(datasets)
fig, ax = plt.subplots(ncols=n_orbits, nrows=1, sharey=True)

for i in range(n_orbits):
    data = fits.getdata(prefix + datasets[i] + '_flt.fits')
    ax[i].imshow(data, origin='lower', vmin=0, vmax=5)
    ax[i].annotate(xy=(600, 950), text=datasets[i], color='white')
    ax[i].set_xlabel('Dipersion pixel')
    if i == 0:
        ax[i].set_ylabel('Cross-dispersion pixel')
    ax[i].set_ylim(a2center[i] - 100, a2center[i] + 100)
    ax[i].set_xlim(250, 500)
    ax[i].axhspan(ymin=a2center[i] - extract_width/2, # Extraction window
                  ymax=a2center[i] + extract_width/2, color='C3', alpha=0.3, label='Extraction')
    ax[i].axhspan(ymin=a2center[i] - bg_offset - bg_size/2, # Background 1 window
                  ymax=a2center[i] - bg_offset + bg_size/2, color='C4', alpha=0.3, label='Background')
    ax[i].axhspan(ymin=a2center[i] + bg_offset - bg_size/2, # Background 2 window
                  ymax=a2center[i] + bg_offset + bg_size/2, color='C4', alpha=0.3)
plt.tight_layout()
plt.legend()

Finally, we extract the spectra at the positions above using the handy `stis_analysis.extract()` function.

**CAUTION**: In the code below, we set `overwrite=True`, which overwrites the `_x1d.fits` files already existent in the output folder. If you do not want to overwrite them, use a different output folder or set `overwrite=False`, which will throw an exception in case the file already exists.

In [ ]:
for i in range(n_orbits):
    stis_analysis.extract(dataset=datasets[i], 
                          prefix=prefix,
                          output_dir=prefix,
                          a2center=a2center[i],
                          extraction_size=extract_width,
                          background1_size=bg_size,
                          background2_size=bg_size,
                          background1_offset=bg_offset,
                          background2_offset=bg_offset,
                          overwrite=True
                      )

And here are the extracted spectra.

In [ ]:
y_plot_scale = 1E-14
for i in range(n_orbits):
    spec = fits.getdata(prefix + datasets[i] + '_x1d.fits')
    wl = spec['WAVELENGTH'][0]
    f = spec['FLUX'][0]
    err = spec['ERROR'][0]
    plt.errorbar(wl, f / y_plot_scale, yerr=err / y_plot_scale, fmt='o', mec='white', ls='-', alpha=0.3, color='C'+str(i))
    plt.plot(wl, f / y_plot_scale, lw='2', color='C'+str(i), label=datasets[i])
plt.xlabel(r'Wavelength [${\rm \AA}$]')
plt.ylabel(r'Flux density [$\times 10^{%i}$ erg s$^{-1}$ cm$^{-2}$ ${\rm \AA}^{-1}$]' % int((np.log10(y_plot_scale))))
plt.xlim(1210, 1220)
plt.ylim(-0.1, 2.1)

plt.legend()
plt.tight_layout()